In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

import pandas as pd
import numpy as np

import requests
import time
import random
from tqdm.notebook import tqdm

import glob

In [2]:
options = Options()

In [3]:
options.add_argument("--ignore-certificate-errors")
options.add_argument("--incognito")

In [4]:
url_filename = 'urls.txt'

In [5]:
if (not url_filename in glob.glob('*.*')):

    driver = webdriver.Chrome("chromedriver.exe", options=options)
    
    url = f'https://www.bookrix.com/books;fantasy,id:52.html'
    driver.get(url)
    time.sleep(5)
    
    book_urls = []
    num_pages = int(driver.find_element(by = 'class name', value = 'pager').text.split('\n')[-1])
    
    
    current_page = 1
    
    while (current_page <= num_pages):
    
        books = driver.find_elements(by = 'class name', value = 'item')
        
        for book in books:
            url = book.find_element(by = 'class name', value = 'word-break').get_attribute('href')
            book_urls.append(url)
        
        pager = driver.find_element(by = 'class name', value = 'pager')
        pager.find_element(by = 'class name', value = 'next').click()
        
        random_wait = random.randrange(5, 15)
        time.sleep(random_wait)
        
        current_page += 1

    with open("urls.txt", 'w') as f:
        for s in book_urls:
            f.write(str(s) + '\n')
    
    driver.close()
            
else:
    with open(url_filename, 'r') as f:
        book_urls = [line.rstrip('\n') for line in f]

In [6]:
books_data = []

In [7]:
columns = ['title', 'author', 'summary', 'category', 'word_count',
           'age_cat', 'num_views', 'num_fav', 'publication_date',
           'tags', 'url', 'download_link']

df_filename = 'books.csv'

if (df_filename in glob.glob('*.*')):
    df_books = pd.read_csv(df_filename, index_col = 0)
else:
    df_books = pd.DataFrame(books_data, columns=columns)

In [8]:
num_to_collect = 300

In [9]:
collected = 0

for url in tqdm(book_urls):
    try:
        if (collected >= num_to_collect):
            break
        if (url in list(df_books['url'])):
            continue
        
        driver = webdriver.Chrome("chromedriver.exe", options=options)
        driver.get(url)
        
        random_wait = random.randrange(10, 20)
        time.sleep(random_wait)
        
        title_box = driver.find_element(by = 'id', value = 'booktitle')
        title = title_box.find_element(by = 'class name', value = 'break-word').text
        author = title_box.find_element(by = 'id', value = 'authortext').text.split(': ')[1]
        summary = driver.find_element(by = 'id', value = 'blurb').text
        
        details = driver.find_element(by = 'id', value = 'details')
        
        details_top = [text.text for text in details.find_elements(by = 'tag name', value = 'li')]
        category = details_top[0]
        word_count = details_top[1]
        age_cat = details_top[2]
        num_views = details_top[3]
        num_fav = details_top[4]
        publication_date = details_top[5].split(': ')[1]
        
        try:
            tags = set(details.find_element(by = 'id', value = 'bookkeywords').text.split('\n')[1].split(', '))
            
        except Exception as e:
            tags = np.nan
        
        download_boxes = driver.find_element(by = 'id', value = 'bookfree')
        
        download_button = download_boxes.find_element(by = 'tag name', value = 'span')
        
        if (not download_button.find_element(by = 'tag name', value = 'a').get_attribute('class').split(' ')[-1] == 'disabled'):
            download_button.click()
            
            random_wait = random.randrange(10, 15)
            time.sleep(random_wait)
            
            download_link = driver.find_element(by = 'class name', value = 'bxmodal-backdrop').find_element(by = 'class name', value = 'modal-body').find_element(by = 'tag name', value = 'a').get_attribute('href')
        else:
            download_link = np.nan
            
        book_info = []
        book_info.append(title)
        book_info.append(author)
        book_info.append(summary)
        book_info.append(category)
        book_info.append(word_count)
        book_info.append(age_cat)
        book_info.append(num_views)
        book_info.append(num_fav)
        book_info.append(publication_date)
        book_info.append(tags)
        book_info.append(url)
        book_info.append(download_link)
        
        books_data.append(book_info)
        
        driver.close()
        
        collected += 1
    except Exception as e:
        pass

  0%|          | 0/3686 [00:00<?, ?it/s]

C:\Users\alcob\AppData\Local\Temp/ipykernel_21432/3657312954.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe", options=options)


In [10]:
df_collected = pd.DataFrame(books_data, columns=columns)
df_collected

,title,author,summary,category,word_count,age_cat,num_views,num_fav,publication_date,tags,url,download_link
0,Magical mermaid: Aqua,Mewichigo34,,Fantasy,121 Words,Ages 0 and up,434,7,09-12-2010,NaN,https://www.bookrix.com/_ebook-mewichigo34-mag...,https://www.bookrix.com/Books/Download.html?bo...
1,Teentopia,Stephanie95,A young lady named Lila gets transported to a ...,Fantasy,6536 Words,Ages 0 and up,418,5,05-27-2010,NaN,https://www.bookrix.com/_ebook-stephanie95-tee...,https://www.bookrix.com/Books/Download.html?bo...
2,Anything Can Happen,Chelsea Bullitt,a girl finds a secret wand anf she wishes for ...,Fantasy,190 Words,Ages 0 and up,281,1,09-21-2011,NaN,https://www.bookrix.com/_ebook-chelsea-bullitt...,https://www.bookrix.com/Books/Download.html?bo...
3,The Hidden World.,Ashleigh Elliott,This book is about a girl who realises that th...,Fantasy,909 Words,Ages 12 and up,278,0,07-25-2014,NaN,https://www.bookrix.com/_ebook-ashleigh-elliot...,https://www.bookrix.com/Books/Download.html?bo...
4,The Sins That Define Us,Lejin Dellinger,Marie Lovelace hunts every kind of evil. Werew...,Fantasy,3 Words,Ages 16 and up,266,10,07-26-2012,"{death, love, killer, hunter, supernatural}",https://www.bookrix.com/_ebook-lejin-dellinger...,https://www.bookrix.com/Books/Download.html?bo...
...,...,...,...,...,...,...,...,...,...,...,...,...
178,Atlas Online:,Nick Venom,The second season of Atlas Online follows Tin ...,Fantasy,6468 Words,Ages 10 and up,18,0,08-06-2021,"{Action, Atlas Online, Adventure,... Show more...",https://www.bookrix.com/_ebook-nick-venom-atla...,https://www.bookrix.com/Books/Download.html?bo...
179,The Great Agony,Frunz Studio,"""The Great Agony"" - such term was chosen by th...",Fantasy,237 Words,Ages 14 and up,14,0,04-02-2021,NaN,https://www.bookrix.com/_ebook-frunz-studio-th...,https://www.bookrix.com/Books/Download.html?bo...
180,Girl of his dream,Presh Bliss,This unfortunate situation inevitably shaped h...,Fantasy,3447 Words,Ages 0 and up,8,1,07-11-2021,NaN,https://www.bookrix.com/_ebook-presh-bliss-gir...,https://www.bookrix.com/Books/Download.html?bo...
181,The Angaran Chronicles: Of Frozen Death and My...,BAD Agar,Alathis and his master are on their first assi...,Fantasy,4065 Words,Ages 12 and up,9,0,09-12-2021,"{dwarf, diesel punk, short story, vampire, vam...",https://www.bookrix.com/_ebook-bad-agar-the-an...,https://www.bookrix.com/Books/Download.html?bo...


In [11]:
df_books = df_books.append(df_collected, ignore_index = True)
df_books

,title,author,summary,category,word_count,age_cat,num_views,num_fav,publication_date,tags,url,download_link
0,My Personal Hell,D. Richardson,"For seventeen year old Ailith, grief and p...",Fantasy,168571 Words,Ages 12 and up,48035,422,10-09-2013,"{'cruelty', 'love', 'werewolves', 'torture', '...",https://www.bookrix.com/_ebook-d-richardson-my...,NaN
1,Marked House of Night,Kristin Cast,A girl is a vampire and goes to school. At a v...,Fantasy,1635 Words,Ages 0 and up,25090,413,12-06-2011,NaN,https://www.bookrix.com/_ebook-kristin-cast-ma...,https://www.bookrix.com/Books/Download.html?bo...
2,Pack Gems,"Kuro Kusai, WhiteWolf Aura",Taylor and Kaden fight fang and claw to keep t...,Fantasy,79586 Words,Ages 12 and up,24830,647,04-02-2011,"{'Pack', 'Shapeshifter,... Show more', 'Fantas...",https://www.bookrix.com/_ebook-kuro-kusai-pack...,https://www.bookrix.com/Books/Download.html?bo...
3,SPELLS OF MAGIC,Forest Ostrander,These spells are actual spells. I would highly...,Fantasy,7957 Words,Ages 0 and up,19129,95,03-04-2012,NaN,https://www.bookrix.com/_ebook-forest-ostrande...,https://www.bookrix.com/Books/Download.html?bo...
4,Marked (book 1),Kristin Cast,The House of Night series is set in a world ve...,Fantasy,82309 Words,Ages 6 and up,17210,52,10-28-2014,{'fantasy'},https://www.bookrix.com/_ebook-kristin-cast-ma...,https://www.bookrix.com/Books/Download.html?bo...
...,...,...,...,...,...,...,...,...,...,...,...,...
2442,Atlas Online:,Nick Venom,The second season of Atlas Online follows Tin ...,Fantasy,6468 Words,Ages 10 and up,18,0,08-06-2021,"{Action, Atlas Online, Adventure,... Show more...",https://www.bookrix.com/_ebook-nick-venom-atla...,https://www.bookrix.com/Books/Download.html?bo...
2443,The Great Agony,Frunz Studio,"""The Great Agony"" - such term was chosen by th...",Fantasy,237 Words,Ages 14 and up,14,0,04-02-2021,NaN,https://www.bookrix.com/_ebook-frunz-studio-th...,https://www.bookrix.com/Books/Download.html?bo...
2444,Girl of his dream,Presh Bliss,This unfortunate situation inevitably shaped h...,Fantasy,3447 Words,Ages 0 and up,8,1,07-11-2021,NaN,https://www.bookrix.com/_ebook-presh-bliss-gir...,https://www.bookrix.com/Books/Download.html?bo...
2445,The Angaran Chronicles: Of Frozen Death and My...,BAD Agar,Alathis and his master are on their first assi...,Fantasy,4065 Words,Ages 12 and up,9,0,09-12-2021,"{dwarf, diesel punk, short story, vampire, vam...",https://www.bookrix.com/_ebook-bad-agar-the-an...,https://www.bookrix.com/Books/Download.html?bo...


In [12]:
df_books.to_csv(f'books.csv')

In [13]:
# r = requests.get(download_link, allow_redirects=True)
#open(f'Books/{title}.epub', 'wb').write(r.content)